In [2]:
import pandas as pd

# Load the datasets
train = pd.read_csv('../kaggle_data/train.csv')
weather = pd.read_csv('../kaggle_data/weather.csv')


In [3]:
#Convert date column of both datasets to datetime
train['Date'] = pd.to_datetime(train['Date'])
weather['Date'] = pd.to_datetime(weather['Date'])

#Print the datatypes of the columns
print(train.dtypes)

Date                      datetime64[ns]
Address                           object
Species                           object
Block                              int64
Street                            object
Trap                              object
AddressNumberAndStreet            object
Latitude                         float64
Longitude                        float64
AddressAccuracy                    int64
NumMosquitos                       int64
WnvPresent                         int64
dtype: object


In [8]:
# Merge the datasets on the 'Date' column
combined_data = pd.merge(train, weather, on='Date')


In [9]:
# Create time-based features
combined_data['Month'] = combined_data['Date'].dt.month
combined_data['Week'] = combined_data['Date'].dt.isocalendar().week
combined_data['DayOfWeek'] = combined_data['Date'].dt.dayofweek


### Combining distance to spraying locations as well

In [11]:
# Load the spray data
from geopy.distance import geodesic
import numpy as np

def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).miles

# Calculate the distance from each trap to the nearest spraying location
min_distances = []
total_rows = len(combined_data)

for index, row in combined_data.iterrows():
    distances = [calculate_distance(row['Latitude'], row['Longitude'], lat, lon) for lat, lon in zip(spray['Latitude'], spray['Longitude'])]
    min_distances.append(min(distances))
    
    # Print progress
    progress = (index + 1) / total_rows * 100
    print(f"Progress: {progress:.2f}%")

combined_data['DistanceToSpray'] = min_distances


Progress: 0.00%
Progress: 0.01%
Progress: 0.01%
Progress: 0.02%
Progress: 0.02%
Progress: 0.03%
Progress: 0.03%
Progress: 0.04%
Progress: 0.04%
Progress: 0.05%
Progress: 0.05%
Progress: 0.06%
Progress: 0.06%
Progress: 0.07%
Progress: 0.07%
Progress: 0.08%
Progress: 0.08%
Progress: 0.09%
Progress: 0.09%
Progress: 0.10%
Progress: 0.10%
Progress: 0.10%
Progress: 0.11%
Progress: 0.11%
Progress: 0.12%
Progress: 0.12%
Progress: 0.13%
Progress: 0.13%
Progress: 0.14%
Progress: 0.14%
Progress: 0.15%
Progress: 0.15%
Progress: 0.16%
Progress: 0.16%
Progress: 0.17%
Progress: 0.17%
Progress: 0.18%
Progress: 0.18%
Progress: 0.19%
Progress: 0.19%
Progress: 0.20%
Progress: 0.20%
Progress: 0.20%
Progress: 0.21%
Progress: 0.21%
Progress: 0.22%
Progress: 0.22%
Progress: 0.23%
Progress: 0.23%
Progress: 0.24%
Progress: 0.24%
Progress: 0.25%
Progress: 0.25%
Progress: 0.26%
Progress: 0.26%
Progress: 0.27%
Progress: 0.27%
Progress: 0.28%
Progress: 0.28%
Progress: 0.29%
Progress: 0.29%
Progress: 0.30%
Progress

In [ ]:
# Example: Creating a feature for the average temperature 3 days before the current date
combined_data['Tavg_3days_before'] = combined_data.groupby('Station')['Tavg'].transform(lambda x: x.rolling(3, min_periods=1).mean().shift())
